In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import numpy as np

data = pd.read_csv("IMS_Dataset.csv")
label_encoder_customer = LabelEncoder()
label_encoder_customer.fit(data['Customer_ID'])
label_encoder_cluster = LabelEncoder()
label_encoder_cluster.fit(data['Cluster_ID'])
data['Customer_ID_encoded'] = label_encoder_customer.transform(data['Customer_ID'])
data['Cluster_ID_encoded'] = label_encoder_cluster.transform(data['Cluster_ID'])

In [4]:
def train_predict_requirement(requirement_column, all_columns):
    predictions = pd.DataFrame(columns=['Customer_ID', 'Cluster_ID', requirement_column + '_Predicted'])
    print("checkpoint")
    for customer_id in data['Customer_ID'].unique():
        customer_data = data[data['Customer_ID'] == customer_id]
        X = customer_data[all_columns]
        y = customer_data[requirement_column]
        model = RandomForestRegressor()
        model.fit(X, y)
        requirement_predicted = model.predict(X)
        mean_prediction = np.mean(requirement_predicted)
        customer_cluster = customer_data['Cluster_ID'].iloc[0]
        customer_prediction = pd.DataFrame({'Customer_ID': [customer_id], 
                                            'Cluster_ID': [customer_cluster],
                                            requirement_column + '_Predicted': [mean_prediction]})
        predictions = pd.concat([predictions, customer_prediction], ignore_index=True)
    return predictions

In [5]:
all_columns = ['HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 'LSHS_Requirement', 'MS_Requirement', 'SKO_Requirement', 'Customer_ID_encoded', 'Cluster_ID_encoded']
fuel_requirements_predictions = pd.DataFrame()
for requirement_column in ['HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 'LSHS_Requirement', 'MS_Requirement', 'SKO_Requirement']:
    prediction_df = train_predict_requirement(requirement_column, all_columns)
    fuel_requirements_predictions = pd.concat([fuel_requirements_predictions, prediction_df], ignore_index=True)

checkpoint
checkpoint
checkpoint
checkpoint
checkpoint
checkpoint


In [6]:
fuel_requirements_predictions.to_csv("Final_Fuel_Requirement_Predictions.csv", index=False)